## Steps for Image Classification Using FastAI :
1. Import required libraries

    * FastAI
    * Numpy
    * Pandas
    * Matplotlib
    * Os    


2. Load and view your data

    * Setting up path for training data
    * Data Loading For training
    * Data Explorations
    * Print Classes present in the data
    
    
3. Create and train a model

     * Create Models
     * Train Model
 
 
4. Interpret the results
5. Prediction Using Trained Model


## 1.  Importing Required Libraries

In [ ]:
# Import Required Python Packages :

import warnings
warnings.filterwarnings('ignore')

# Setting up our enviroment
# Data Viz & Regular Expression Libraries :
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Scientific and Data Manipulation Libraries :
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import FastAI library 
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate
import os

## 2.  Load and view your data
### 2.1  Setting up path for training data

#### Fun Fact: Number of elements in a list of path is same as number of classes you have


In [ ]:
x  = '/kaggle/input/cat-and-dog/training_set/training_set'
path = Path(x)
path.ls()


### 2.2  Data Loading For training

Things to be remember:

* Decide validation percentage ( 0.2 => 20% )
* Provide path for training data
* Decide augmentations criteria (optional)
* Decide image size (which is 224 in my case)
* Test data can also be added but it's optional

In [ ]:

np.random.seed(40)
data = ImageDataBunch.from_folder(path, train = '.', valid_pct=0.2,
                                  ds_tfms=get_transforms(), size=224,
                                  num_workers=4).normalize(imagenet_stats)

### 2.3  Data Explorations
**Our image dataset is stored as .jpg files in 2 different folders, with each folder bearing the name of model of the images contained in the folder. We use the ImageDataBunch.from_folder() function to load the images and assign labels the images based on the name of the folder they’re read from.**

In [ ]:
data.show_batch(rows=4, figsize=(7,6),recompute_scale_factor=True)

In [ ]:
data

### 2.4  Print Classes present in the data
data.c — How many classes are there in our dataset?

len(data.train_ds) — What is the size of our training dataset?

len(data.valid_ds) — What is the size of our validation dataset?**

In [ ]:

print(data.classes)
len(data.classes)
data.c

## 3.   Create and train a model

* We now use a pre-trained ResNet18 Convolutional Neural Net model, and use transfer learning to learn weights of only the last layer of the network.

* Why Transfer learning? Because with transfer learning, you begin with an existing (trained) neural network used for image recognition — and then tweak it a bit (or more) here and there to train a model for your particular use case. And why do we do that? Training a reasonable neural network would mean needing approximately 300,000 image samples, and to achieve really good performance, we’re going to need at least a million images.

* In our case, we have approximately 2500 images in our training set — you have one guess to decide if that would have been enough if were to train a neural net from scratch.

* We use the **create_cnn()** function for loading a pre-trained ResNet18 network, that was trained on around a million images from the ImageNet database.


### 3.1. Create Model

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy], model_dir = Path('../kaggle/working'),path = Path("."))


### 3.2  Training Neural Network

To find the perfect learning rates we can use the lr_find and recorder.plot methods which create a plot that relates the learning rate with the loss.

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestions=True)

In [ ]:
lr1 = 1e-3
lr2 = 1e-1
learn.fit_one_cycle(4,slice(lr1,lr2))

### woooho (95% + accuracy) All hail to [Jeremy howard](https://en.wikipedia.org/wiki/Jeremy_Howard_(entrepreneur)[](http://))

## 4.  Interpret the results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

## 5. Prediction using trained model

In [ ]:
img = open_image('../input/cat-and-dog/test_set/test_set/dogs/dog.4003.jpg')
print(learn.predict(img)[0])
img

## Deploying your model (OPTIONAL)

When you are ready to put your model in production, export the minimal state of your Learner with:

In [ ]:
learn.export(file = Path("/kaggle/working/export.pkl"))


In [ ]:
learn.model_dir = "/kaggle/working"
learn.save("stage-1",return_path=True)

## Sources
* [Fastai MOOC](https://course.fast.ai/)
* [Fastai library](http://https://docs.fast.ai/)
